<span style="color:#8B949E;">
<b>Note de lecture</b> — Notebook issu de tests itératifs (“speed-tests”).  
Le corpus utilisé ici est un <b>dataset de substitution</b> (non aligné client) uniquement pour valider la mécanique (retrieval + évaluation) et dérouler la roadmap.  
Pour le chemin complet, suivre l’ordre 01 → 10 et lire en priorité les sections Markdown.
</span>


# 🧪 Script 10 — Benchmark Hybride sur corpus chunké (JSONL)  
### BM25 • Dense • Hybride **RRF** (BM25 + Dense)

## 🧭 Pourquoi ce stage (progression logique)
Après :
- **Stage 8** : construction d’un corpus **chunké** (XML → JSONL) pour obtenir une granularité exploitable,
- **Stage 9** : benchmark BM25 **sur chunks** (baseline plus réaliste que sur documents entiers),

➡️ ce **Stage 10** vise à comparer, sur **exactement le même corpus chunké** et le même benchmark :

- 🧾 **BM25** (lexical)
- 🧠 **Dense retrieval** (embeddings)
- 🧩 **Hybride via RRF** : fusion de deux classements indépendants (**BM25 rank** + **Dense rank**) via *Reciprocal Rank Fusion*

🎯 Objectif : **mesurer l’apport réel de l’hybride**, tout en gardant une logique “audit-ready” :
- le retrieval **benché** est le même que celui affiché en **verbatim** (top-k),
- l’oracle de pertinence **V2** (article-aware via `meta.num`) est utilisé de manière cohérente.

---

## ✅ Ce que ce notebook contient
- ⚙️ Un bloc “setup notebook” (install + `sys.path`) pour éviter les erreurs en Jupyter
- 🧾 Le **code original** du script (non réécrit ; seulement encapsulé en cellule)
- 🧾 Une cellule Markdown séparée avec la **conclusion / bilan** (détachée de la fin de script)

> 📌 Note : le script utilise des chemins Windows par défaut (Spyder).  
> Dans un notebook, on peut soit :
> - lancer `main()` tel quel **après avoir ajusté** les arguments/chemins,
> - ou exécuter le script en ligne de commande avec `--corpus-jsonl`, `--cache-dir`, etc.


In [1]:
# 🧱 Dépendances (à exécuter une seule fois par environnement)
# Si tu es déjà dans un venv où tout est installé, tu peux sauter cette cellule.

%pip -q install rank-bm25 sentence-transformers numpy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 🔧 Import de modules locaux (si le notebook n'est pas à la racine du projet)
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd()  # ajuste en .parent si besoin (ex: notebook dans /notebooks)
sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)


PROJECT_ROOT: d:\-- Projet RAG Avocats --\codes_python\notebooks


## ▶️ Exécution (2 options)

### Option A — exécuter `main()` dans le notebook
Dans le code, `main()` force quelques paramètres “IDE-friendly” (verbatim, query understanding, filtres).  
Tu peux donc simplement exécuter la cellule du script, puis appeler :

```python
main()
```

⚠️ Pense à **adapter** les chemins par défaut dans `parse_args()` si ton JSONL/cache ne sont pas au même endroit.

### Option B — exécuter en ligne de commande (reproductible)
Exemple (à adapter) :

```bash
python 10_benchmark_hybride_rrf_bm25_dense_chunks.py --retriever hybrid --k 10 --doc-types article --collection-contains LEGITEXT000006072050
```


In [4]:
# -*- coding: utf-8 -*-
"""
SCRIPT 10 — BENCHMARK RETRIEVAL SUR CORPUS CHUNKÉ (JSONL) :
BM25, Dense, et Hybride BM25 + Dense via RRF (Reciprocal Rank Fusion)

But du script
-------------
Ce script bench un moteur de retrieval sur un corpus juridique déjà chunké (format JSONL).
Il compare trois stratégies :
- BM25 (lexical) sur texte chunké,
- Dense retrieval (embeddings Sentence-Transformers) sur tous les chunks,
- Hybride RRF : fusion de deux rankings (BM25 et Dense) via Reciprocal Rank Fusion.

Le script a été stabilisé pour que :
- le retrieval évalué (benchmark) soit exactement le même que celui affiché en mode verbatim,
- l’oracle de pertinence V2 soit utilisé de manière cohérente (article-aware).

Corpus & filtrage
-----------------
Le corpus est chargé depuis un JSONL (1 ligne = 1 chunk).
Un filtrage optionnel réduit le corpus à une sous-collection et à des types de documents (doc_type),
par ex. sur le Code du travail :
- collection_contains="LEGITEXT000006072050"
- doc_types=["article"]

Dense embeddings & cache
------------------------
Les embeddings sont calculés (ou rechargés) avec SentenceTransformer
(ex: "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2").
Un cache .npy est utilisé, avec une clé qui inclut les paramètres qui impactent le corpus filtré
(chemin corpus, modèle, doc_types, title_contains, collection_contains, limit).
Une vérification bloque l’exécution si le cache ne correspond pas au nombre de chunks filtrés
(anti-désalignement).

Hybride RRF
-----------
L’hybride ne fait pas un simple “BM25 shortlist -> rerank dense”.
Il produit deux rankings indépendants :
- ranking BM25 (sur tokens normalisés + stopwords FR),
- ranking Dense (cosine similarity via embeddings normalisés),
puis les fusionne via RRF :
score(doc) = Σ 1 / (rrf_k + rank_i(doc))
Le top-k final provient du ranking fusionné.

Évaluation (oracle V2)
----------------------
Le benchmark utilise un oracle V2 “article-aware” (via benchmark_queries_v2) :
- si meta.num est présent et qu’on a des préfixes attendus, un chunk est pertinent
  uniquement si meta.num commence par un des préfixes ;
- le fallback mots-clés n’est utilisé que si meta.num est absent (ou si aucun préfixe n’est défini).
Un fallback V1 (keywords via metrics.py) reste possible uniquement si V2 n’est pas importable.

Sorties
-------
- Métriques : Recall@k, MRR, nDCG@k (pertinence binaire, compatible POC).
- Mode verbatim : affiche le top-k par requête avec doc_id, meta.num et le flag PERTINENT/NON PERTINENT
  pour audit qualitatif du pipeline réellement benché.

Constat sur le dernier test (référence)
---------------------------------------
Avec filtrage Code du travail + doc_types=article (~386 chunks), en hybride RRF :
- la query "licenciement pour motif économique" est correctement retrouvée (L1233-3),
- les queries "CDI rompu sans préavis" et "contester un licenciement" ne retombent pas dans les préfixes attendus,
d’où un plateau de métriques à 0.333 sur un benchmark de 3 questions.
Ce résultat reflète le comportement réel du retrieval (plus un artefact de scoring/verbatim).

Référence fichiers : script testé et log du dernier run = resultats_script_10_dernier_test.txt 
"""


import argparse
import hashlib
import json
import os
import re
from typing import Any, Dict, List, Optional, Sequence, Tuple
import unicodedata  # Normalisation simple (accents) pour une tokenisation BM25 plus stable

import numpy as np
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer

from query_understanding import load_juridical_dictionary, process_user_query
import math  # Utilisé pour nDCG (log2)

# On garde metrics.py uniquement comme fallback (oracle keywords) si jamais V2 n’est pas dispo
import metrics as kw_metrics  # recall_at_k / reciprocal_rank / ndcg_at_k basés sur mots-clés

# ---------------------------------------------------------------------
# Benchmark queries (V2 oracle meta.num) avec fallback
# ---------------------------------------------------------------------
try:
    # Cas recommandé : benchmark_queries_v2.py expose une fonction de génération
    from benchmark_queries_v2 import get_benchmark_queries_v2  # type: ignore

    BENCHMARK_QUERIES = get_benchmark_queries_v2()  # Liste d’objets ou de dicts selon mon implémentation
    USING_V2 = True  # Flag utilisé dans l’évaluation + verbatim

except Exception:
    try:
        # Cas simple : benchmark_queries_v2.py expose directement une liste (ex: benchmark_queries_v2 = [...])
        from benchmark_queries_v2 import benchmark_queries_v2  # type: ignore

        BENCHMARK_QUERIES = benchmark_queries_v2  # Alias unique utilisé partout
        USING_V2 = True  # Flag utilisé dans l’évaluation + verbatim

    except Exception:
        # Fallback ultime : ancien benchmark keywords (si tu veux garder une compatibilité)
        from benchmark_queries import get_benchmark_queries  # type: ignore

        BENCHMARK_QUERIES = get_benchmark_queries()  # Liste de dicts (question + relevant_keywords)
        USING_V2 = False  # On reste sur oracle keywords



# ---------------------------------------------------------------------
# 1) IO — Chargement JSONL
# ---------------------------------------------------------------------

def load_chunks_jsonl(jsonl_path: str, limit: Optional[int] = None) -> List[Dict[str, Any]]:
    """Charge un JSONL (1 ligne = 1 chunk dict)."""
    if not os.path.exists(jsonl_path):
        raise FileNotFoundError(f"Fichier introuvable: {jsonl_path}")

    chunks: List[Dict[str, Any]] = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except json.JSONDecodeError:
                continue
            if "doc_id" not in obj or "text" not in obj:
                continue
            chunks.append(obj)
            if limit is not None and len(chunks) >= limit:
                break
    return chunks


# ---------------------------------------------------------------------
# 2) Filtrage métier (optionnel)
# ---------------------------------------------------------------------

def chunk_matches_title(chunk: Dict[str, Any], needle: str) -> bool:
    """Match sur meta.titre (sinon fallback doc_id)."""
    needle_low = (needle or "").lower().strip()
    if not needle_low:
        return True
    meta = chunk.get("meta") or {}
    titre = (meta.get("titre") or "").lower()
    doc_id = (chunk.get("doc_id") or "").lower()
    return (needle_low in titre) or (needle_low in doc_id)


def filter_chunks(
    chunks: Sequence[Dict[str, Any]],
    allowed_doc_types: Optional[Sequence[str]] = None,
    title_contains: Optional[str] = None,
    collection_contains: Optional[str] = None,
) -> List[Dict[str, Any]]:
    """Filtre par doc_type + sous-chaîne dans titre/doc_id + sous-chaîne de collection dans doc_id."""
    allowed = None if allowed_doc_types is None else {t.lower().strip() for t in allowed_doc_types}  # Normalise la whitelist
    needle_collection = (collection_contains or "").lower().strip() or None  # Normalise le filtre “collection”
    out: List[Dict[str, Any]] = []

    for c in chunks:
        if allowed is not None:
            dt = (c.get("doc_type") or "").lower()  # Normalise doc_type
            if dt not in allowed:  # Filtre doc_type
                continue

        if title_contains:
            if not chunk_matches_title(c, title_contains):  # Filtre métier existant
                continue

        if needle_collection:
            doc_id = (c.get("doc_id") or "").lower()  # Fallback simple et stable car doc_id est toujours présent
            if needle_collection not in doc_id:  # Filtre “collection” par chemin
                continue

        out.append(c)

    return out



# ---------------------------------------------------------------------
# 3) BM25 — pour mode hybride (shortlist)
# ---------------------------------------------------------------------

# ---------------------------------------------------------------------
# Stopwords FR (liste volontairement courte et stable pour un POC)
# ---------------------------------------------------------------------

STOPWORDS_FR = {
    # Articles / déterminants
    "le", "la", "les", "un", "une", "des", "du", "de", "d", "au", "aux", "ce", "cet", "cette", "ces",
    # Pronoms / relatifs
    "je", "tu", "il", "elle", "on", "nous", "vous", "ils", "elles", "qui", "que", "quoi", "dont", "où",
    # Prépositions / conjonctions courantes
    "a", "à", "en", "dans", "sur", "sous", "chez", "par", "pour", "avec", "sans", "entre", "vers",
    "et", "ou", "mais", "donc", "or", "ni", "car",
    # Verbes très fréquents (bruit pour BM25)
    "est", "sont", "etre", "être", "avoir", "peut", "peuvent", "doit", "doivent",
    # Question / formulation (bruit)
    "qu", "que", "quels", "quelles", "quel", "quelle", "quand", "comment", "pourquoi",
    "cas", "faire", "fait", "faits", "afin",
    # Divers
    "plus", "moins", "tres", "très", "tout", "toute", "tous", "toutes",
}


def normalize_for_bm25(text: str) -> str:
    """
    Normalise un texte pour BM25 :
    - minuscule
    - suppression des accents (équilibre pratique/robustesse en POC)
    - uniformisation d’apostrophes/espaces
    """
    text = (text or "").lower()  # Uniformise la casse pour éviter des doublons lexicaux
    text = text.replace("’", "'")  # Uniformise apostrophes typographiques (fréquentes en juridique)
    text = unicodedata.normalize("NFKD", text)  # Décompose lettres+accents pour pouvoir retirer les accents
    text = "".join(ch for ch in text if not unicodedata.combining(ch))  # Retire les diacritiques (accents)
    text = re.sub(r"\s+", " ", text).strip()  # Compacte les espaces (plus stable pour regex)
    return text


def tokenize(text: str) -> List[str]:
    """
    Tokenisation BM25 robuste :
    - normalisation (accents, casse)
    - extraction de tokens alphanumériques
    - filtrage stopwords
    - filtrage tokens trop courts (réduit fortement les faux positifs)
    """
    normalized = normalize_for_bm25(text)  # Normalise texte (mêmes règles pour requête et corpus)
    raw_tokens = re.findall(r"\b\w+\b", normalized)  # Extrait tokens “mots” (simple et debug-friendly)
    tokens: List[str] = []
    for tok in raw_tokens:
        if len(tok) < 3:  # Ignore “a”, “de”, “du”… et autres tokens courts très bruités
            continue
        if tok.isdigit():  # Évite de surpondérer des numéros isolés (souvent du bruit)
            continue
        if tok in STOPWORDS_FR:  # Retire stopwords (évite shortlist polluée par “cas”, “sans”, etc.)
            continue
        tokens.append(tok)
    return tokens



def build_bm25_index(documents: Sequence[Dict[str, Any]]) -> BM25Okapi:
    """Construit un index BM25 sur documents[i]['text']."""
    return BM25Okapi([tokenize(d["text"]) for d in documents])


def bm25_search(
    query: str,
    documents: Sequence[Dict[str, Any]],
    bm25: BM25Okapi,
    top_k: int
) -> List[Tuple[int, float]]:
    """
    Retourne une liste (doc_index, score) BM25 triée, sur top_k.
    On retourne des index pour pouvoir reranker dense proprement.
    """
    scores = bm25.get_scores(tokenize(query))
    ranked = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]

def hybrid_rrf_search(
    query: str,
    documents: Sequence[Dict[str, Any]],
    bm25: BM25Okapi,
    model: SentenceTransformer,
    doc_emb: np.ndarray,
    top_k: int,
    shortlist_k: int = 200,
    rrf_k: int = 60,
) -> List[Tuple[Dict[str, Any], float]]:
    """
    Fusion BM25 + Dense via RRF (Reciprocal Rank Fusion).
    Objectif : éviter que le dense 'casse' BM25, tout en ajoutant du signal sémantique.
    """
    # Top indices BM25 (shortlist)
    bm25_top = bm25_search(query, documents, bm25, top_k=shortlist_k)  # [(idx, score), ...]
    bm25_idx = [i for i, _ in bm25_top]  # On ne garde que les rangs (plus stable que les scores bruts)

    # Top indices Dense (shortlist)
    q_emb = model.encode([query], normalize_embeddings=True).astype(np.float32)[0]  # Embedding requête normalisé
    dense_scores = doc_emb @ q_emb  # Similarités cosinus (dot product sur vecteurs normalisés)
    dense_idx = np.argsort(-dense_scores)[:shortlist_k].tolist()  # Rang dense

    # Fusion RRF sur indices
    fused: Dict[int, float] = {}  # idx_doc -> score_rff

    for rank, idx in enumerate(bm25_idx, start=1):
        fused[idx] = fused.get(idx, 0.0) + 1.0 / (rrf_k + rank)  # Contribution BM25 par rang

    for rank, idx in enumerate(dense_idx, start=1):
        fused[idx] = fused.get(idx, 0.0) + 1.0 / (rrf_k + rank)  # Contribution Dense par rang

    # Tri final
    best = sorted(fused.items(), key=lambda x: x[1], reverse=True)[:top_k]  # Top-k fusionnés
    return [(documents[idx], float(score)) for idx, score in best]  # Retour (doc, score)




# ---------------------------------------------------------------------
# 4) Dense — embeddings + search (avec cache)
# ---------------------------------------------------------------------

def _cache_key(*parts: str) -> str:
    """Fabrique une clé stable pour le cache embeddings."""
    h = hashlib.md5()
    for p in parts:
        h.update(p.encode("utf-8"))
    return h.hexdigest()


def build_dense_embeddings(
    documents: Sequence[Dict[str, Any]],
    model_name: str,
    cache_dir: str,
    cache_tag: str,
) -> Tuple[SentenceTransformer, np.ndarray]:
    """
    Construit (ou recharge) la matrice d'embeddings des chunks.
    - Normalisation activée => cos-sim = produit scalaire.
    """
    os.makedirs(cache_dir, exist_ok=True)
    cache_path = os.path.join(cache_dir, f"dense_emb_{cache_tag}.npy")

    model = SentenceTransformer(model_name)

    if os.path.exists(cache_path):
        emb = np.load(cache_path)
        # Sécurité : si le cache ne correspond pas au corpus courant, on le reconstruit
        if emb.shape[0] != len(documents):  # Nombre de vecteurs != nombre de chunks filtrés
            # On reconstruit pour éviter de reranker sur des embeddings d’un autre corpus
            os.remove(cache_path)  # Supprime le cache incohérent pour forcer un rebuild propre
            emb = model.encode(
                [d.get("text", "") for d in documents],     # Embeddings calculés exactement sur docs filtrés
                batch_size=64,                              # Batch raisonnable pour CPU
                show_progress_bar=True,                     # Progress bar utile en POC
                normalize_embeddings=True                   # Normalisation => cos-sim = dot product
            ).astype(np.float32)                            # float32 pour perf/mémoire
            np.save(cache_path, emb)                        # Sauvegarde du nouveau cache cohérent
            return model, emb                               # Retourne modèle + embeddings alignés



    texts = [d.get("text", "") for d in documents]
    emb = model.encode(
        texts,
        batch_size=64,
        show_progress_bar=True,
        normalize_embeddings=True
    ).astype(np.float32)

    np.save(cache_path, emb)
    return model, emb


def dense_search_all(
    query: str,
    documents: Sequence[Dict[str, Any]],
    model: SentenceTransformer,
    doc_emb: np.ndarray,
    top_k: int
) -> List[Tuple[Dict[str, Any], float]]:
    """Dense search sur tous les documents (cosine similarity)."""
    q_emb = model.encode([query], normalize_embeddings=True).astype(np.float32)[0]
    scores = doc_emb @ q_emb  # (N,)
    idx = np.argsort(-scores)[:top_k]
    return [(documents[i], float(scores[i])) for i in idx]


def dense_rerank_subset(
    query: str,
    documents: Sequence[Dict[str, Any]],
    model: SentenceTransformer,
    doc_emb: np.ndarray,
    subset_idx: Sequence[int],
    top_k: int
) -> List[Tuple[Dict[str, Any], float]]:
    """Rerank dense sur un sous-ensemble d'indices (mode hybride)."""
    q_emb = model.encode([query], normalize_embeddings=True).astype(np.float32)[0]
    sub = np.array(list(subset_idx), dtype=np.int64)
    scores = doc_emb[sub] @ q_emb  # (M,)
    order = np.argsort(-scores)[:top_k]
    best = sub[order]
    return [(documents[i], float(scores[j])) for j, i in zip(order, best)]


# ---------------------------------------------------------------------
# 5) Query text — brut / enrichi / minimal métier
# ---------------------------------------------------------------------

def minimal_business_expansion(query: str) -> str:
    """
    Expansion ultra-minimale, “métier”, sans dépendre du dictionnaire.
    Objectif : aider BM25/dense sur certaines requêtes RH typiques.

    Note : c’est volontairement petit, pour rester POC.
    """
    q = query.lower()
    extra = []

    if "sans préavis" in q or ("rompu" in q and "préavis" in q):
        extra += ["faute grave", "faute lourde", "prise d'acte", "démission", "rupture"]
    if "contester" in q and "licenciement" in q:
        extra += ["prud'hommes", "délai", "contestations", "cause réelle et sérieuse", "nullité"]

    if not extra:
        return query
    return query + " " + " ".join(extra)


def _safe_lower(s: str) -> str:
    """Lowercase sûr (évite None)."""
    return (s or "").lower()

def _q_get(q: Any, key: str, default: Any = None) -> Any:
    """
    Accès robuste aux champs des queries.
    - Supporte dict (q["question"]) OU objet/dataclass (q.question).
    """
    if isinstance(q, dict):  # Cas dict
        return q.get(key, default)
    return getattr(q, key, default)  # Cas objet (dataclass / class)


def q_id(q: Any) -> str:
    """Retourne l’ID de la query (qid / id)."""
    return str(_q_get(q, "qid", _q_get(q, "id", "")))


def q_question(q: Any) -> str:
    """Retourne la question (texte) de la query."""
    return str(_q_get(q, "question", ""))


def q_num_prefixes(q: Any) -> List[str]:
    """Retourne la liste des préfixes meta.num attendus (oracle V2)."""
    prefixes = _q_get(q, "relevant_num_prefixes", [])
    return list(prefixes) if prefixes else []


def q_keywords_fallback(q: Any) -> List[str]:
    """
    Retourne la liste de mots-clés fallback.
    - En V2 : relevant_keywords_fallback
    - En V1 : relevant_keywords (oracle keywords)
    """
    kw = _q_get(q, "relevant_keywords_fallback", None)
    if kw is None:
        kw = _q_get(q, "relevant_keywords", [])
    return list(kw) if kw else []


def is_relevant_v2(doc: Dict[str, Any], q: Any) -> bool:
    """
    Oracle V2 : pertinence principalement par meta.num (préfixes attendus).
    Le fallback par mots-clés ne doit servir que si meta.num est manquant (sinon on réintroduit du bruit type V1).

    Règles
    ------
    1) Si on a des préfixes attendus ET que meta.num est présent :
       - pertinent uniquement si meta.num commence par un des préfixes.
       - sinon NON pertinent (on ne tente pas le fallback keywords).
    2) Si meta.num est absent (ou si aucun préfixe n'est défini) :
       - fallback : pertinent si le texte contient un des mots-clés fallback.
    """
    meta = doc.get("meta") or {}
    num = (meta.get("num") or "").strip()

    prefixes = q_num_prefixes(q)

    # Cas "article-aware" strict : meta.num existe et on sait quelles zones on attend
    if num and prefixes:
        for p in prefixes:
            if num.startswith(p):
                return True
        return False  # meta.num est présent mais hors-zone => on refuse (pas de fallback keywords)

    # Fallback uniquement si meta.num est absent (ou si prefixes est vide)
    kw_fallback = q_keywords_fallback(q)
    if kw_fallback:
        text = _safe_lower(doc.get("text", ""))
        for kw in kw_fallback:
            if _safe_lower(kw) in text:
                return True

    return False



def recall_at_k_v2(results: List[Tuple[Dict[str, Any], float]], q: Any, k: int) -> float:
    """
    Recall@k (V2) :
    - 1 si au moins un chunk pertinent (oracle V2) est présent dans le top-k
    - 0 sinon
    """
    for doc, _ in results[:k]:
        if is_relevant_v2(doc, q):
            return 1.0
    return 0.0


def reciprocal_rank_v2(results: List[Tuple[Dict[str, Any], float]], q: Any) -> float:
    """
    MRR (V2) :
    - 1 / rang du premier chunk pertinent
    - 0 si aucun chunk pertinent
    """
    for rank, (doc, _) in enumerate(results, start=1):
        if is_relevant_v2(doc, q):
            return 1.0 / float(rank)
    return 0.0


def ndcg_at_k_v2(results: List[Tuple[Dict[str, Any], float]], q: Any, k: int) -> float:
    """
    nDCG@k (V2) en pertinence binaire.

    On reproduit l’esprit de metrics.py :
    - DCG : somme(1/log2(rank+1)) sur les chunks pertinents trouvés dans le top-k
    - IDCG : DCG idéal avec le même nombre de pertinents “trouvés” remontés en tête
    """
    dcg = 0.0
    found_rels = 0  # Nombre de pertinents trouvés dans le top-k

    for rank, (doc, _) in enumerate(results[:k], start=1):
        if is_relevant_v2(doc, q):
            dcg += 1.0 / math.log2(rank + 1)  # Gain binaire (1) discounté par le rang
            found_rels += 1  # On compte les pertinents trouvés

    if found_rels == 0:
        return 0.0  # Aucun pertinent => nDCG nul

    idcg = 0.0
    for i in range(1, found_rels + 1):
        idcg += 1.0 / math.log2(i + 1)  # Classement idéal : pertinents aux premiers rangs

    return float(dcg / idcg) if idcg > 0 else 0.0


def get_query_variant(
    base_query: str,
    dictionary: Optional[Dict[str, Any]],
    use_query_understanding: bool,
    use_min_expansion: bool,
) -> str:
    """Construit la requête effective (brute / enrichie / minimale métier)."""
    q = base_query
    if use_query_understanding and dictionary is not None:
        enriched = process_user_query(base_query, dictionary)
        q = enriched.get("enriched_query", base_query)

    if use_min_expansion:
        q = minimal_business_expansion(q)

    return q


# ---------------------------------------------------------------------
# 6) Benchmark — générique (BM25 / dense / hybride)
# ---------------------------------------------------------------------

def evaluate_retriever(
    retriever_name: str,
    documents: Sequence[Dict[str, Any]],
    k: int,
    dictionary: Optional[Dict[str, Any]],
    use_query_understanding: bool,
    use_min_expansion: bool,
    bm25: Optional[BM25Okapi] = None,
    bm25_shortlist: int = 200,
    model: Optional[SentenceTransformer] = None,
    doc_emb: Optional[np.ndarray] = None,
) -> Dict[str, float]:
    """Calcule Recall@k, MRR, nDCG@k sur benchmark_queries pour un retriever donné."""
    recall_scores: List[float] = []
    mrr_scores: List[float] = []
    ndcg_scores: List[float] = []

    for q in BENCHMARK_QUERIES:

        qtext = get_query_variant(
            base_query=q_question(q),
            dictionary=dictionary,
            use_query_understanding=use_query_understanding,
            use_min_expansion=use_min_expansion,
        )

        if retriever_name == "bm25":
            if bm25 is None:
                raise ValueError("bm25 requis pour retriever=bm25")
            results = [(doc, score) for doc, score in [(documents[i], s) for i, s in bm25_search(qtext, documents, bm25, top_k=k)]]

        elif retriever_name == "dense":
            if model is None or doc_emb is None:
                raise ValueError("model + doc_emb requis pour retriever=dense")
            results = dense_search_all(qtext, documents, model, doc_emb, top_k=k)

        elif retriever_name == "hybrid":
            # Hybride = fusion BM25 + dense via RRF.
            # On garde exactement le même chemin que celui audité en verbatim (voir show_verbatim),
            # sinon on "débuggue" un pipeline différent de celui réellement scoré.
            if bm25 is None or model is None or doc_emb is None:
                raise ValueError("bm25 + model + doc_emb requis pour retriever=hybrid")
            results = hybrid_rrf_search(
                query=qtext,                 # Requête finale (brute/enrichie)
                documents=documents,          # Corpus filtré
                bm25=bm25,                    # Index BM25
                model=model,                  # Modèle dense
                doc_emb=doc_emb,              # Embeddings docs
                top_k=k,                      # Top-k affiché
                shortlist_k=bm25_shortlist,    # Taille shortlist interne
                rrf_k=60,                     # Constante RRF standard POC
            )


        else:
            raise ValueError(f"retriever inconnu: {retriever_name}")

        if USING_V2:
            # Oracle V2 : pertinence par meta.num + fallback keywords
            recall_scores.append(recall_at_k_v2(results, q, k))
            mrr_scores.append(reciprocal_rank_v2(results, q))
            ndcg_scores.append(ndcg_at_k_v2(results, q, k))
        else:
            # Fallback : ancien oracle keywords (metrics.py)
            relevant_keywords = q_keywords_fallback(q)  # Sur V1, c’est relevant_keywords
            recall_scores.append(kw_metrics.recall_at_k(results, relevant_keywords, k))
            mrr_scores.append(kw_metrics.reciprocal_rank(results, relevant_keywords))
            ndcg_scores.append(kw_metrics.ndcg_at_k(results, relevant_keywords, k))



    return {
        f"Recall@{k}": float(sum(recall_scores) / len(recall_scores)),
        "MRR": float(sum(mrr_scores) / len(mrr_scores)),
        f"nDCG@{k}": float(sum(ndcg_scores) / len(ndcg_scores)),
    }


def show_verbatim(
    retriever_name: str,
    documents: Sequence[Dict[str, Any]],
    k: int,
    dictionary: Optional[Dict[str, Any]],
    use_query_understanding: bool,
    use_min_expansion: bool,
    bm25: Optional[BM25Okapi] = None,
    bm25_shortlist: int = 200,
    model: Optional[SentenceTransformer] = None,
    doc_emb: Optional[np.ndarray] = None,
    preview_chars: int = 450,
) -> None:
    """Affiche top-k par requête pour audit qualitatif."""
    for q in BENCHMARK_QUERIES:
        qtext = get_query_variant(
            base_query=q_question(q),
            dictionary=dictionary,
            use_query_understanding=use_query_understanding,
            use_min_expansion=use_min_expansion,
        )

        print("\n" + "=" * 110)
        print(f"ID: {q_id(q)} | Question: {q_question(q)}")
        print(f"Query effective: {qtext}")
        print("=" * 110)

        if retriever_name == "bm25":
            ranked_idx = bm25_search(qtext, documents, bm25, top_k=k)
            results = [(documents[i], float(s)) for i, s in ranked_idx]
            score_label = "BM25"
        elif retriever_name == "dense":
            results = dense_search_all(qtext, documents, model, doc_emb, top_k=k)
            score_label = "DenseSim"
        elif retriever_name == "hybrid":
            # IMPORTANT : alignement strict verbatim <-> benchmark.
            # Le benchmark "hybrid" utilise RRF (BM25 + dense). Le verbatim doit afficher la même chose.
            if bm25 is None or model is None or doc_emb is None:
                raise ValueError("bm25 + model + doc_emb requis pour retriever=hybrid")
            results = hybrid_rrf_search(
                query=qtext,
                documents=documents,
                bm25=bm25,
                model=model,
                doc_emb=doc_emb,
                top_k=k,
                shortlist_k=bm25_shortlist,
                rrf_k=60,
            )
            score_label = "RRF"
        else:
            raise ValueError(retriever_name)

        for rank, (doc, score) in enumerate(results, start=1):
            text = doc.get("text", "")
            meta = doc.get("meta") or {}
            doc_type = doc.get("doc_type")
            chunk_index = doc.get("chunk_index")
            
            if USING_V2:
                # Oracle V2 : pertinence par meta.num (préfixes) puis fallback keywords.
                is_rel = is_relevant_v2(doc, q)
            else:
                # Oracle V1 : pertinence par présence de mots-clés dans le texte (metrics.py).
                relevant_keywords = q_keywords_fallback(q)
                is_rel = kw_metrics.is_relevant(doc.get("text", ""), relevant_keywords)

            flag = "PERTINENT" if is_rel else "NON PERTINENT"

            print(f"\n--- Rang {rank} | Score {score_label}: {score:.3f} | {flag}")
            print(f"doc_type={doc_type} | chunk_index={chunk_index} | meta.id={meta.get('id')} | meta.num={meta.get('num')}")
            print(f"doc_id: {doc.get('doc_id')}")
            print(text[:preview_chars].replace("\n", " "))


# ---------------------------------------------------------------------
# 7) Main — valeurs par défaut compatibles Spyder
# ---------------------------------------------------------------------

def parse_args() -> argparse.Namespace:
    """Arguments CLI (defaults pour run Spyder)."""
    p = argparse.ArgumentParser(add_help=True)

    p.add_argument("--corpus-jsonl", default=r"D:\-- Projet RAG Avocats --\data_main\result_tests\corpus_chunks.jsonl")
    p.add_argument("--dictionary-yml", default="juridical_dictionary.yml")
    p.add_argument("--k", type=int, default=10)

    p.add_argument("--doc-types", default="article")
    p.add_argument("--title-contains", default="")

    p.add_argument("--limit", type=int, default=None)

    p.add_argument("--retriever", choices=["bm25", "dense", "hybrid"], default="hybrid")
    p.add_argument("--embedding-model", default="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    p.add_argument("--cache-dir", default=r"D:\-- Projet RAG Avocats --\data_main\result_tests\cache_dense")
    p.add_argument("--bm25-shortlist", type=int, default=200)

    p.add_argument("--use-query-understanding", action="store_true")
    p.add_argument("--min-expansion", action="store_true")

    p.add_argument("--show-verbatim", action="store_true")
    
    # LEGITEXT000006072050 semble correspondre à "code du travail" référence (à confirmer)
    p.add_argument("--collection-contains", default="", help="Filtre collection : sous-chaîne à trouver dans doc_id (ex: LEGITEXT000006072050). Vide => pas de filtre.")


    return p.parse_args()


def main() -> None:
    args = parse_args()

    # Réglages “IDE-friendly”
    args.show_verbatim = True  # Audit qualitatif top-k
    args.use_query_understanding = True  # Requête enrichie via dictionnaire
    args.min_expansion = False  # Expansion minimale (si tu la gardes dans le script)
    args.collection_contains = "LEGITEXT000006072050"  # Code du travail (stable dans doc_id)
    args.doc_types = "article"



    doc_types = [x.strip() for x in (args.doc_types or "").split(",") if x.strip()]
    title_contains = (args.title_contains or "").strip() or None

    print("\n=== Chargement corpus chunké (JSONL) ===")
    chunks = load_chunks_jsonl(args.corpus_jsonl, limit=args.limit)
    print(f"Chunks chargés : {len(chunks)}")

    print("\n=== Filtrage (optionnel) ===")
    docs = filter_chunks(
        chunks,
        allowed_doc_types=doc_types if doc_types else None,
        title_contains=title_contains,
        collection_contains=(args.collection_contains or "").strip() or None,  # Active filtre si non vide
    )

    print(f"Chunks après filtre : {len(docs)}")
    # Affiche le filtre collection utilisé (utile pour comprendre pourquoi on passe à 386 docs)
    if (args.collection_contains or "").strip():
        print(f"Filtre collection_contains : {args.collection_contains!r}")  # Debug explicite

    if title_contains:
        print(f"Filtre titre_contains : {title_contains!r}")
    if doc_types:
        print(f"Filtre doc_types : {doc_types}")

    if not docs:
        raise RuntimeError("Aucun chunk après filtrage. Ajuste doc_types / title_contains.")

    dictionary = load_juridical_dictionary(args.dictionary_yml)

    bm25 = None
    model = None
    doc_emb = None

    if args.retriever in ("bm25", "hybrid"):
        print("\n=== Construction index BM25 ===")
        bm25 = build_bm25_index(docs)
        print("Index BM25 construit.")

    if args.retriever in ("dense", "hybrid"):
        print("\n=== Construction / chargement embeddings (dense) ===")
        # On récupère la valeur du filtre "collection" (si présent) pour l’utiliser partout de façon cohérente
        collection_contains = (args.collection_contains or "").strip() or ""  # Vide => pas de filtre

        # La clé de cache DOIT inclure tous les paramètres qui changent le corpus "docs" final
        tag = _cache_key(
            os.path.abspath(args.corpus_jsonl),             # Chemin corpus (impacte le contenu)
            args.embedding_model,                           # Modèle embedding (impacte le vecteur)
            ",".join(doc_types),                            # Doc types retenus (impacte docs)
            title_contains or "",                           # Filtre titre (impacte docs)
            collection_contains,                            # Filtre collection (impacte docs)  <-- CRITIQUE
            str(args.limit or ""),                          # Limite (impacte docs)
        )

        model, doc_emb = build_dense_embeddings(
            documents=docs,
            model_name=args.embedding_model,
            cache_dir=args.cache_dir,
            cache_tag=tag,
        )
        print(f"Embeddings prêts: {doc_emb.shape}")
        
        # Sécurité absolue : si ça ne matche pas, on arrête tout
        if doc_emb.shape[0] != len(docs):
            raise ValueError(
                "Embeddings désalignés avec le corpus filtré : "
                f"len(docs)={len(docs)} vs doc_emb.shape[0]={doc_emb.shape[0]}. "
                "Le cache dense ne correspond pas aux filtres courants (doc_types/titre/collection/limit)."
            )

    print(f"\n=== Benchmark ({args.retriever}) — brute ===")
    res_raw = evaluate_retriever(
        retriever_name=args.retriever,
        documents=docs,
        k=args.k,
        dictionary=dictionary,
        use_query_understanding=False,
        use_min_expansion=False,
        bm25=bm25,
        bm25_shortlist=args.bm25_shortlist,
        model=model,
        doc_emb=doc_emb,
    )
    for kk, vv in res_raw.items():
        print(f"{kk} : {vv:.3f}")

    print(f"\n=== Benchmark ({args.retriever}) — query understanding ===")
    res_enriched = evaluate_retriever(
        retriever_name=args.retriever,
        documents=docs,
        k=args.k,
        dictionary=dictionary,
        use_query_understanding=True,
        use_min_expansion=False,
        bm25=bm25,
        bm25_shortlist=args.bm25_shortlist,
        model=model,
        doc_emb=doc_emb,
    )
    for kk, vv in res_enriched.items():
        print(f"{kk} : {vv:.3f}")

    print(f"\n=== Benchmark ({args.retriever}) — expansion minimale métier ===")
    res_minexp = evaluate_retriever(
        retriever_name=args.retriever,
        documents=docs,
        k=args.k,
        dictionary=dictionary,
        use_query_understanding=False,
        use_min_expansion=True,
        bm25=bm25,
        bm25_shortlist=args.bm25_shortlist,
        model=model,
        doc_emb=doc_emb,
    )
    for kk, vv in res_minexp.items():
        print(f"{kk} : {vv:.3f}")

    if args.show_verbatim:
        print(f"\n=== VERBATIM ({args.retriever}) — brute ===")
        show_verbatim(
            retriever_name=args.retriever,
            documents=docs,
            k=args.k,
            dictionary=dictionary,
            use_query_understanding=False,
            use_min_expansion=False,
            bm25=bm25,
            bm25_shortlist=args.bm25_shortlist,
            model=model,
            doc_emb=doc_emb,
        )

        print(f"\n=== VERBATIM ({args.retriever}) — expansion minimale métier ===")
        show_verbatim(
            retriever_name=args.retriever,
            documents=docs,
            k=args.k,
            dictionary=dictionary,
            use_query_understanding=False,
            use_min_expansion=True,
            bm25=bm25,
            bm25_shortlist=args.bm25_shortlist,
            model=model,
            doc_emb=doc_emb,
        )


if __name__ == "__main__":
    import sys
    sys.argv = [sys.argv[0]]

    main()


=== Chargement corpus chunké (JSONL) ===
Chunks chargés : 13180

=== Filtrage (optionnel) ===
Chunks après filtre : 386
Filtre collection_contains : 'LEGITEXT000006072050'
Filtre doc_types : ['article']

=== Construction index BM25 ===
Index BM25 construit.

=== Construction / chargement embeddings (dense) ===


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Embeddings prêts: (386, 384)

=== Benchmark (hybrid) — brute ===
Recall@10 : 0.333
MRR : 0.333
nDCG@10 : 0.333

=== Benchmark (hybrid) — query understanding ===
Recall@10 : 0.333
MRR : 0.333
nDCG@10 : 0.333

=== Benchmark (hybrid) — expansion minimale métier ===
Recall@10 : 0.333
MRR : 0.333
nDCG@10 : 0.333

=== VERBATIM (hybrid) — brute ===

ID: Q1 | Question: Dans quels cas un CDI peut-il être rompu sans préavis ?
Query effective: Dans quels cas un CDI peut-il être rompu sans préavis ?

--- Rang 1 | Score RRF: 0.024 | NON PERTINENT
doc_type=article | chunk_index=1 | meta.id=None | meta.num=L1251-43
doc_id: D:\-- Projet RAG Avocats --\data_main\data\20250731-220719\legi\global\code_et_TNC_en_vigueur\code_en_vigueur\LEGI\TEXT\00\00\06\07\20\LEGITEXT000006072050\article\LEGI\ARTI\00\00\06\90\12\LEGIARTI000006901299.xml
5° La nature des équipements de protection individuelle que le salarié utilise. Il précise, le cas échéant, si ceux-ci sont fournis par l'entreprise de travail temporaire

## 🧾 Conclusion / Bilan (détaché)

BILAN FINAL — Script 10 (BM25 + Dense + Hybride RRF) : état des lieux après corrections

Références :
- Script 10 corrigé : alignement benchmark/verbatim + oracle V2 strict (meta.num) + doc_types=article. :contentReference[oaicite:0]{index=0}
- Résultats du dernier test : filtre Code du travail + articles uniquement + verbatim détaillé. :contentReference[oaicite:1]{index=1}

1) Ce qui est désormais "verrouillé" et sain (objectif du mini-patch atteint)
- Une seule logique de vérité terrain utilisée partout en V2 : pertinence = meta.num commence par un préfixe attendu ;
  le fallback keywords n'intervient que si meta.num est absent. (Plus de faux positifs type V1 déguisés.)
- Une seule fonction de retrieval hybride utilisée partout : HYBRID = RRF(BM25_rank, Dense_rank) pour benchmark + verbatim.
- Cache dense rendu cohérent : la clé inclut les filtres (doc_types/titre/collection/limit) et un garde-fou stoppe si désalignement.

2) Lecture des résultats du dernier run (important : on interprète un benchmark à 3 questions)
- Corpus : 13 180 chunks chargés, filtrés à 386 (collection_contains='LEGITEXT000006072050', doc_types=['article']). :contentReference[oaicite:2]{index=2}
- Embeddings : (386, 384) => cohérents avec le corpus filtré. :contentReference[oaicite:3]{index=3}
- Scores (hybrid, k=10) :
  * brute        : Recall@10=0.333, MRR=0.333, nDCG@10=0.333
  * query_understanding : identique
  * min_expansion : identique
  => Sur 3 queries, cela signifie : 1 seule query a un "hit" pertinent dans le top-10, et ce hit est rang 1.

3) Diagnostic qualitatif (verbatim)
- Q2 (licenciement économique) : succès net.
  L1233-3 remonte rang 1..8 et est jugé PERTINENT => oracle V2 valide, retrieval cohérent. :contentReference[oaicite:4]{index=4}
- Q1 (rompre un CDI sans préavis) : échec au sens de l’oracle V2.
  Le top-10 ne contient pas d'article dont meta.num commence par les préfixes attendus (ex: L1234*),
  et les résultats sont hors-sujet (L1251-43, L3142-17, etc.). :contentReference[oaicite:5]{index=5}
- Q3 (contester un licenciement) : échec au sens de l’oracle V2.
  Le top-10 est massivement attiré par L1233-3 (économique), même après expansion "prud'hommes / délai / nullité". :contentReference[oaicite:6]{index=6}

4) Conclusion : le "plateau à 0.333" est maintenant EXPLICABLE et NON ambigü
- Ce n’est plus un artefact de mix V1/V2 ou de verbatim décalé.
- On observe un vrai phénomène : avec ce corpus filtré (386 articles) et ces 3 requêtes,
  l’hybride RRF retrouve correctement la définition du licenciement économique (Q2),
  mais ne récupère pas les zones attendues pour "sans préavis" (Q1) et "contester" (Q3).

5) Causes probables (sans lancer de nouveaux tests, juste lecture logique des sorties)
- "Sans préavis" et "contester un licenciement" sont des intentions juridiques qui, dans le Code,
  peuvent se formuler via des notions/conditions (faute grave, préavis, rupture, prescription/délai, contestation prud’homale)
  qui ne matchent pas assez bien les formulations présentes dans les chunks filtrés OU qui sont dispersées ailleurs.
- Sur Q3, l’attraction par L1233-3 indique un fort signal lexical/sémantique autour de "licenciement"
  qui domine le ranking ; sans étape de "query routing" / réécriture ciblée / contraintes structurelles,
  l’hybride peut rester bloqué sur le mauvais chapitre.
- Query understanding et min_expansion n’apportent pas d’amélioration => leur enrichissement actuel n’est pas assez discriminant
  ou pas aligné avec les cibles oracle (préfixes meta.num attendus).

6) Ce que nous validons définitivement à ce stade (pour la roadmap / entretien)
- Le script 10 est devenu un banc d’essai propre : retrieval évalué = retrieval affiché ; oracle V2 strict = métriques fiables.
- Les scores bas restants reflètent un problème "produit" (stratégie retrieval/QA) et/ou "données" (structure/metadata),
  pas un bug d’évaluation.
- Implication directe pour une mise en production RAG juridique :
  il faut une étude systématique du corpus (inventaire balises/structures XML, qualité métadonnées meta.num,
  versions/dates, typologies, évolution temporelle) afin de définir une stratégie d’extraction/chunking
  + un retrieval robuste (routing par typologie, contraintes par livre/titre, etc.).